Import Spark libraries to Start a Spark Session

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StructType,StringType,IntegerType
spark=SparkSession.builder.appName("Filtering,Slicing and View Creation").getOrCreate()

Read Data with Schema Changes

In [11]:
df=spark.read.json('/Users/solansah/Desktop/Data/people.json',schema= StructType([StructField('age',IntegerType(),True),
                                                                      StructField('name',StringType(),True)])
                   )
df.show()
df.describe().show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [12]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



Select Columns